Demo - Training Separate vs Shared Networks

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Spandan-Madan/generalization_biased_category_pose/blob/main/demos/separate_vs_shared.ipynb)

In [6]:
!git clone https://github.com/Spandan-Madan/generalization_biased_category_pose.git

fatal: destination path 'generalization_biased_category_pose' already exists and is not an empty directory.


In [7]:
import os
def create_folder(path):
    if not os.path.isdir(path):
        os.mkdir(path)

create_folder('generalization_biased_category_pose/data')
create_folder('generalization_biased_category_pose/data/mnist_rotation_data')

In [8]:
!wget https://www.dropbox.com/s/wdws3b3fjo190sk/self_generated.tar.gz?dl=0 -O generalization_biased_category_pose/data/mnist_rotation_data/self_generated.tar.gz

--2021-03-03 01:49:56--  https://www.dropbox.com/s/wdws3b3fjo190sk/self_generated.tar.gz?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wdws3b3fjo190sk/self_generated.tar.gz [following]
--2021-03-03 01:49:57--  https://www.dropbox.com/s/raw/wdws3b3fjo190sk/self_generated.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbcbd55ba49c0bdf31877d6d25f.dl.dropboxusercontent.com/cd/0/inline/BJ-1dPwX0TPpXBRLH5HcoaxX-l0kv0EwWsv3ZD7zXsCHvqvOccKplJ_9YkQ9ZpLRtizI7fbDKUlgDwTvJRfjp3WxrzIkcBDl7BrnEuAVLAGwE9uB3vJ37YUlDh5_sdCs2YM/file# [following]
--2021-03-03 01:49:57--  https://ucbcbd55ba49c0bdf31877d6d25f.dl.dropboxusercontent.com/cd/0/inline/BJ-1dPwX0TPpXBRLH5HcoaxX-l0kv0EwWsv3ZD7zXsCHvqvOccKplJ_9YkQ9ZpLRtizI7fbDKUlgDwT

In [9]:
!cd generalization_biased_category_pose/data/mnist_rotation_data/ && tar -zxf self_generated.tar.gz

In [10]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
import random
ImageFile.LOAD_TRUNCATED_IMAGES = True
import argparse
import pickle
import sys
sys.path.append('generalization_biased_category_pose/res/')
from models.models import get_model
from loader.loader import get_loader

/content
generalization_biased_category_pose/res/loader/multi_attribute_loader.py
generalization_biased_category_pose/res/loader
generalization_biased_category_pose/res/loader/loader.py
generalization_biased_category_pose/res/loader


In [11]:
from tqdm.notebook import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("poster")
sns.set_palette("Set1", 8, .75)
sns.despine()

<Figure size 432x288 with 0 Axes>

In [47]:
DATASET_NAME = 'mnist_rotation_six_by_nine'
NUM_EPOCHS = 10
BATCH_SIZE = 200
ARCH = 'LATE_BRANCHING_COMBINED'

image_transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])


GPU = 1

In [13]:
curr_dir = os.getcwd()
repo_dir = '/'.join(curr_dir.split('/')[:-1])
data_dir = "%s/data/"%repo_dir

In [14]:
if 'ilab' in DATASET_NAME:
    NUM_CLASSES = (6,6,6,6)
elif 'uiuc' in DATASET_NAME:
    NUM_CLASSES = (8,8,8,8)
elif 'mnist_rotation' in DATASET_NAME:
    NUM_CLASSES = (10,10,10,10)
else:
    NUM_CLASSES = (5,5,4,5)

if 'ilab' in DATASET_NAME:
    loader_new = get_loader('multi_attribute_loader_file_list_ilab')
elif 'uiuc' in DATASET_NAME:
    loader_new = get_loader('multi_attribute_loader_file_list_uiuc')
elif 'mnist_rotation' in DATASET_NAME:
    loader_new = get_loader('multi_attribute_loader_file_list_mnist_rotation')
else:
    loader_new = get_loader('multi_attribute_loader_file_list')

file_list_root = 'generalization_biased_category_pose/dataset_lists/'
att_path = 'generalization_biased_category_pose/dataset_lists/combined_attributes.p'
# loader_new = get_loader('multi_attribute_loader_file_list_ilab')

In [15]:
shuffles = {'train':True,'val':True,'test':False}

In [16]:
data_dir = 'generalization_biased_category_pose/data/'

In [17]:
################ GET FROM USER CONFIG - TODO #####################
file_lists = {}
dsets = {}
dset_loaders = {}
dset_sizes = {}
for phase in ['train','val','test']:
    file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,DATASET_NAME)
    dsets[phase] = loader_new(file_lists[phase],att_path, image_transform, data_dir)
    dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=BATCH_SIZE, shuffle = shuffles[phase], num_workers=2,drop_last=True)
    dset_sizes[phase] = len(dsets[phase])

In [18]:
multi_losses = [nn.CrossEntropyLoss(),nn.CrossEntropyLoss(),nn.CrossEntropyLoss(),nn.CrossEntropyLoss()]

In [19]:
def weight_scheduler(epoch_num, task):
    if task == 'shared':
        return [0.0,1.0,0.0,1.0]
    elif task == 'viewpoint':
        return [0.0,1.0,0.0,0.0]
    elif task == 'category':
        return [0.0,0.0,0.0,1.0]

In [27]:
def train_epoch(model, task, optimizer):
    model.train()
    torch.set_grad_enabled(True)
    phase = 'train'
    
    weights = weight_scheduler(epoch, task)
    iters = 0
    phase_epoch_corrects = [0,0,0,0]
    phase_epoch_loss = 0
    
    for data in tqdm(dset_loaders[phase]):
        inputs, labels_all, paths = data
        inputs = Variable(inputs.float().cuda())

        optimizer.zero_grad()
        model_outs = model(inputs)
        calculated_loss = 0
        batch_corrects = [0,0,0,0]
        
        for i in range(4):
            labels = labels_all[:,i]
            if GPU:
                labels = Variable(labels.long().cuda())
            loss = multi_losses[i]
            outputs = model_outs[i]
            calculated_loss += weights[i] * loss(outputs,labels)
            _, preds = torch.max(outputs.data, 1)
            batch_corrects[i] = torch.sum(preds == labels.data)
            phase_epoch_corrects[i] += batch_corrects[i]

        
        phase_epoch_loss += calculated_loss
        calculated_loss.backward()
        optimizer.step()
        iters += 1
    epoch_loss = phase_epoch_loss/dset_sizes[phase]
    # print('Train loss:%s'%epoch_loss)
    epoch_accs = [float(i)/dset_sizes[phase] for i in phase_epoch_corrects]

    if task == 'shared':
        epoch_gm = np.sqrt(epoch_accs[1] * epoch_accs[3])
    elif task == 'viewpoint':
        epoch_gm = epoch_accs[1]
    elif task == 'category':
        epoch_gm = epoch_accs[3]
    
    return model, epoch_loss, epoch_gm

In [28]:
def test_epoch(model, best_model, best_test_loss, best_test_gm, task):
    model.eval()
    torch.set_grad_enabled(False)
    phase = 'test'
    weights = weight_scheduler(epoch, task)
    iters = 0
    phase_epoch_corrects = [0,0,0,0]
    phase_epoch_loss = 0
    
    for data in tqdm(dset_loaders[phase]):
        inputs, labels_all, paths = data
        inputs = Variable(inputs.float().cuda())
        model_outs = model(inputs)
        calculated_loss = 0
        batch_corrects = [0,0,0,0]
        
        for i in range(4):
            labels = labels_all[:,i]
            if GPU:
                labels = Variable(labels.long().cuda())
            loss = multi_losses[i]
            outputs = model_outs[i]
            calculated_loss += weights[i] * loss(outputs,labels)
            _, preds = torch.max(outputs.data, 1)
            batch_corrects[i] = torch.sum(preds == labels.data)
            phase_epoch_corrects[i] += batch_corrects[i]


        phase_epoch_loss += calculated_loss
        iters += 1
    epoch_loss = phase_epoch_loss/dset_sizes[phase]
    # print('Test loss:%s'%epoch_loss)
    epoch_accs = [float(i)/dset_sizes[phase] for i in phase_epoch_corrects]
    
    if task == 'shared':
        epoch_gm = np.sqrt(epoch_accs[1] * epoch_accs[3])
    elif task == 'viewpoint':
        epoch_gm = epoch_accs[1]
    elif task == 'category':
        epoch_gm = epoch_accs[3]
    
    if epoch_loss < best_test_loss:
        best_model = model
        best_test_loss = epoch_loss
        best_test_gm = epoch_gm
    
    return best_model, epoch_loss, epoch_gm, best_test_loss, best_test_gm

Separate vs Shared Networks

In [46]:
models = {} 
models['shared']= get_model(ARCH,NUM_CLASSES)
models['viewpoint']= get_model(ARCH,NUM_CLASSES)
models['category']= get_model(ARCH,NUM_CLASSES)

models['shared'].cuda();
models['viewpoint'].cuda();
models['category'].cuda();

best_models = {}
best_models['shared'] = models['shared']
best_models['viewpoint'] = models['viewpoint']
best_models['category'] = models['category']

best_test_loss = 100
best_test_gm = 0

all_train_gms = {}
all_train_gms['shared'] = [0]
all_train_gms['separate'] = [0]

all_test_gms = {}
all_test_gms['shared'] = [0]
all_test_gms['separate'] = [0]

optimizers = {}
optimizers['shared'] = optim.Adam(models['shared'].parameters(), lr=0.001)
optimizers['viewpoint'] = optim.Adam(models['viewpoint'].parameters(), lr=0.001)
optimizers['category'] = optim.Adam(models['category'].parameters(), lr=0.001)

In [ ]:

for epoch in range(10):
    train_gm_shared = 1
    test_gm_shared = 1
    for TASK in ['viewpoint','category','shared']:
        print('Epoch: %s, Task: %s'%(epoch,TASK))
        print('---------')
        models[TASK], train_loss, train_gm = train_epoch(models[TASK], TASK, optimizers[TASK])
        best_models[TASK], test_loss, test_gm, best_test_loss, best_test_gm = test_epoch(models[TASK], best_models[TASK], best_test_loss, best_test_gm, TASK)
        if TASK != 'shared':
            train_gm_shared = train_gm_shared * train_gm
            test_gm_shared = test_gm_shared * test_gm

    all_train_gms['separate'].append(np.sqrt(train_gm_shared))
    all_test_gms['separate'].append(np.sqrt(test_gm_shared))
    all_train_gms['shared'].append(train_gm)
    all_test_gms['shared'].append(test_gm)

    
    clear_output()
    print('Epoch: %s'%epoch)
    print('---------')
    
    
    plt.plot(all_train_gms['separate'], color = 'blue', marker = 'o')
    plt.plot(all_train_gms['shared'], color = 'red', marker = 'o')
    plt.legend(['separate', 'shared'])    
    plt.title('Train Accuracy')
    plt.show()
    
    plt.plot(all_test_gms['separate'], color = 'blue', marker = 'o')
    plt.plot(all_test_gms['shared'], color = 'red', marker = 'o')
    plt.legend(['separate', 'shared'])    
    plt.title('Test Accuracy')
    plt.show()